# Train GAN to generate a song with 100 notes

In [ ]:
! git clone https://github.com/Skuldur/Classical-Piano-Composer.git

Cloning into 'Classical-Piano-Composer'...
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 721.79 MiB | 22.68 MiB/s, done.
Resolving deltas: 100% (41/41), done.


## prepare dataset

In [ ]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
with open('./Classical-Piano-Composer/data/notes', 'rb') as filepath:
    notes = pickle.load(filepath)

In [ ]:
# Get all pitch names
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))

In [ ]:
def prepare_input_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    #normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = numpy.array(network_input) / float(n_vocab)

    return (network_input, normalized_input, numpy.array(output))

In [ ]:
network_input, normalized_input, output = prepare_input_sequences(notes, pitchnames, n_vocab)

In [ ]:
normalized_input.shape

(27790, 100)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((normalized_input))

In [ ]:
train_ds = train_dataset.batch(128, True)

## generator model

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    # Fully connected layer:
    model.add(layers.Dense(100, activation='relu', input_shape=(20,)))

    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    # Layer 3
    model.add(layers.Dense(100, activation='sigmoid'))

    return model

In [ ]:
generator = make_generator_model()

## discriminator model

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(100,)))

    # Fully connected layer
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
discriminator = make_discriminator_model()

## loss function

In [ ]:
# This method returns a helper function to compute the binary cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    generator_loss = cross_entropy(tf.ones_like(fake_output),fake_output)
    return generator_loss

In [ ]:
learning_rate = 1e-5
generator_optimizer = tf.keras.optimizers.Adam(1e-5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)

## training step

In [ ]:
import time
import os

In [ ]:
checkpoint_dir = "song_cp_new"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
batch_size = 128
noise_dim = 20
num_examples_to_generate = 1

sample_noise_vector = tf.random.normal([num_examples_to_generate, noise_dim])
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(songs):
    noise_vector = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      # Generator output
      generated_songs = generator(noise_vector, training=True)

      # Discriminator output
      real_output = discriminator(songs, training=True)
      fake_output = discriminator(generated_songs, training=True)

      # Loss functions
      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    # Gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Update both networks
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def generate_song(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  pred = numpy.reshape(predictions.numpy() * n_vocab, (100,)).astype(int)

  sample_generated_song = []
  for n in pred:
    if n==279:
      n=278
    sample_generated_song.append(int_to_note[n])
  print(sample_generated_song)


In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for song_batch in dataset:
      train_step(song_batch)

    # Save the model every 100 epochs
    if (epoch + 1) % 100 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    generate_song(generator, epoch + 1, sample_noise_vector)

In [ ]:
#%%time
train(train_ds, 100)

Time for epoch 1 is 1.4456756114959717 sec
['D5', 'G#1', 'G1', '1.6', '4.9', '9.0.4', '0.4.7', 'E2', '8.9.1.4', '10.0.3', 'G4', 'G#6', 'G6', '9.11.0.4', 'E-6', 'G4', 'G#6', 'G#4', '11', 'D1', 'G4', 'E5', 'G6', 'G6', 'G6', '4.7.9', 'G3', 'D5', 'F3', 'G4', 'E6', 'F2', '11.4', 'G#2', 'G5', 'E3', 'G6', '4.7.9.0', 'F#6', 'G4', 'G#3', 'G4', 'G#2', '0.6', '9.2', 'B-1', 'G3', 'B6', '2.8', 'G6', 'A0', 'G#4', '8.9.2', 'D6', '5', 'G6', 'G4', 'B-6', 'G4', 'G#5', 'G3', 'F6', 'G5', 'G6', 'A0', 'G4', 'G4', 'G5', 'A2', 'C#1', 'F6', 'G3', 'F#1', 'E-6', 'F2', 'C#1', 'E-3', 'G6', 'G6', '0.2.5', 'G#6', '4.5.9', 'F3', '10.0.4', '8', '10.11', 'G5', '1.4.8', 'G4', 'G6', '2.5.7', 'F#5', '10.2.3', '2.5', 'G6', 'F#1', '2.4.9', 'G2', 'G1', 'F2']
Time for epoch 2 is 1.5374221801757812 sec
['G#2', 'G#4', '1.3.8', 'G4', 'D2', 'E-4', '10.0', '4.8', 'F4', 'E1', 'G#5', 'G6', '6.8', 'G#1', '11.3.6', 'G1', 'G4', 'B-6', 'G2', 'G2', 'G3', '0.4.5', 'G6', 'G6', 'G1', 'G2', 'E-3', 'G5', 'D1', 'G3', 'G#5', 'G6', 'C#2', 'G6', 

## save model

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
checkpoint_dir = './simple_gan_song_cp'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

checkpoint.save(file_prefix = checkpoint_prefix)

'./simple_gan_song_cp/ckpt-1'

In [ ]:
!cp -r  "song_cp_2" "/content/gdrive/My Drive/simple_gan_song_cp_final/"  #move file from A to B

cp: cannot stat 'gan_song_cp_new_2': No such file or directory


## restore model

In [ ]:
!cp -r "/content/gdrive/My Drive/simple_gan_song_cp_final/" "simple_gan_song_cp_final3"  #move file from A to B

In [ ]:
checkpoint_dir = './simple_gan_song_cp_final3'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

status = checkpoint.restore(tf.train.latest_checkpoint('./simple_gan_song_cp_final3'))

## generate song

In [ ]:
def create_midi(prediction_output, i):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    file_name = "gan_output"+str(i)+".mid"

    midi_stream.write('midi', fp=file_name)

In [ ]:
sample_noise_vector = tf.random.normal([num_examples_to_generate, noise_dim])
predictions = generator(sample_noise_vector, training=False)

pred = numpy.reshape(predictions.numpy() * n_vocab, (100,)).astype(int)

sample_generated_song = []
for n in pred:
  if n==279:
    n=278
  sample_generated_song.append(int_to_note[n])
print(sample_generated_song)


['G4', 'G5', 'G6', 'F2', '9.1.4', 'G1', 'G2', 'G6', 'G4', 'G6', 'G6', 'C3', 'G1', 'G2', 'D1', 'G4', 'G1', '4.6.10', '0.3.7', 'G6', 'G#5', 'G5', 'G1', 'G6', 'G5', 'G5', 'G#6', '1', 'G3', '1', '2.5.7', 'F#1', 'G3', 'G2', '1.4.7.9', '3.6.8', 'G2', 'D3', 'G1', 'G#1', 'G6', 'E5', 'G6', 'G3', 'C3', 'C#5', 'D7', 'C4', '2.6.7.9', 'G6', 'G5', '8.11.1.4', 'F#3', 'G4', '4.6', 'G6', '3.5', 'G4', 'G2', 'F#4', 'F6', 'G#6', 'G#2', 'G6', 'G5', 'G4', 'G#3', 'E-5', '3.7', '7.11', '4.5.9.0', 'F6', 'B-5', '6.8', 'F#2', 'A6', 'E1', '8.10.3', 'G#4', 'B-2', 'G5', 'G#4', '9', '6.7.11', 'G#2', 'G4', 'G6', 'F#2', 'G6', 'G#2', '8.10.1', 'G#2', 'G6', 'F5', 'G2', 'C#4', 'G6', 'C#7', 'G4', 'G6']


In [ ]:
create_midi(sample_generated_song, 30)

# GAN generate first 100 notes, then lstm continues with the next 500 notes

In [ ]:
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

In [ ]:
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [ ]:
def prepare_input_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [ ]:
def create_trained_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('./Classical-Piano-Composer/weights.hdf5')

    return model

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction

    pattern = []
    for n in network_input:
      pattern.append(note_to_int[n])
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
network_input, normalized_input = prepare_input_sequences(notes, pitchnames, n_vocab)

In [ ]:
model = create_trained_network(normalized_input, n_vocab)

In [ ]:
prediction_output = generate_notes(model, sample_generated_song, pitchnames, n_vocab)

In [ ]:
generated_song = sample_generated_song + prediction_output

In [ ]:
create_midi(generated_song, 105)